# Loading

Create a generator function for importing the file

In [1]:
import urllib.request
import lzma
import base64
import re

def get_dependencies():
    localfile, info = urllib.request.urlretrieve("https://github.com/kevSweet/meta-deps/raw/master/pypi-deps.csv.lzma")
    with lzma.open(localfile, mode='r') as file:
        for line in file:
            package, version, deps = line.decode("utf-8").split('\t')
            deps = base64.b64decode(deps).decode("utf-8")
            deps = eval(deps)
            regex = r'\"*([\w\.]+)[><]?\=?.*\"*'
            
            def cleanDep(s):
                try:
                    match = re.search(regex, s)
                    return match.group(0)
                except:
                    return None
            
            deps = [cleanDep(x) for x in deps if cleanDep(x) is not None]
            
            yield package, deps
        
        


Store as dataframe

In [2]:
import pandas as pd
import numpy as np

rows = np.array([[None,None]])
for package, deps in get_dependencies():
    # Packages with no dependencies get blank string
    if deps:
        args= [[package, dep] for dep in deps]
    else:
        args = [[package, None]]
    rows = np.append(rows, args, axis=0)

        
df = pd.DataFrame(rows, columns=["package", "dependency"]).drop_duplicates()
df = df.dropna(subset=['package'])
df.head()

,package,dependency
1,0x10c-asm,None
2,2gis,None
3,3to2,None
4,3to2_py3k,None
5,42qucc,None


In [3]:
deps = pd.Series(df.dependency.unique())
packages = pd.Series(df.package.unique())
new_deps = deps[~deps.isin(packages)]
new_df = pd.DataFrame({'package': new_deps, 'dependency': [None]*len(new_deps)})
df = pd.concat([df, new_df])

In [4]:
from timeit import default_timer

## Redshift

In Redshift we create a single table that will hold one "edge" per row.

In [12]:
import redshift_connector
conn = redshift_connector.connect(
     host='redshift-python-dep.cantgzv41ter.us-east-1.redshift.amazonaws.com',
     database='deps',
     user='awsuser',
     password='REDACTED'
  )
conn.autocommit = True
cursor = conn.cursor()

Create the table

In [13]:
cursor.execute("DROP TABLE dependencies")
cursor.execute(
"""
    CREATE TABLE dependencies (
        package VARCHAR(150),
        dependency VARCHAR(150) NULL
    )
"""
)

Load the data

In [ ]:


# Time it
start = default_timer()

cursor.write_dataframe(df, "dependencies")

# for package, deps in get_dependencies():
#     # Packages with no dependencies get blank string
#     if deps:
#         args = list(product([package], deps))
#     else:
#         args = [(package, "")]
        
#     print(args)
    
#     for tup in args:
#         cursor.execute("INSERT INTO dependencies VALUES (%s, %s)", tup)
#     cursor.executemany("INSERT INTO dependencies VALUES (%s, %s)", args)

end = default_timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282

# Neptune Graph

Clear existing data from graph

In [ ]:
%%gremlin

g.V().drop()

Load in nodes

In [ ]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T

graph = Graph()

remoteConn = DriverRemoteConnection('wss://database-1.cluster-c8vhkkn9knug.us-west-2.neptune.amazonaws.com:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)


for package_name in df.package.unique():
    g.add_v('package').property(T.id, package_name).as_(package_name).next()
    print("added package: " + package_name)

remoteConn.close()

In [ ]:
%%gremlin

g.V()

Add edges

In [ ]:
remoteConn = DriverRemoteConnection('wss://database-1.cluster-c8vhkkn9knug.us-west-2.neptune.amazonaws.com:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)


for index, row in df.iterrows():
    print(row['dependency'])
    if row['dependency'] is not None:
        g.V(row['package']).add_e("depends_on").to(row['dependency']).next()
        print("Added dependency: " + row['package'] + "->" + row['dependency'])
        
remoteConn.close()

In [8]:
%%gremlin

g.E()